In [23]:
import cv2
import numpy as np
import glob  # for read file from folder
import time  # for estimate processing time(optional)
import matplotlib.pyplot as plt
import copy


q = '/home/garima/Desktop/clutter/Beer_images/query/*'
d = '/home/garima/Desktop/clutter/Beer_images/database/*'
GOOD_MATCH_THRESH = 10
correct_count = 0
query_images = glob.glob(q)
database_images = glob.glob(d)
query_images

['/home/garima/Desktop/clutter/Beer_images/query/amstel_light.jpg',
 '/home/garima/Desktop/clutter/Beer_images/query/bira_blonde.jpg',
 '/home/garima/Desktop/clutter/Beer_images/query/becks_dark.jpg',
 '/home/garima/Desktop/clutter/Beer_images/query/becks.jpeg']

In [24]:
for query_name in query_images:
    max_count_name = None
    max_count = 0
    query = cv2.imread(query_name, 0)
    # _, q_desc = sift.main(query, verbose=True)
#     _, q_desc = feature_extractor.detectAndCompute(query, None)
    for database_name in database_images:
        database = cv2.imread(query_name, 0)
#         d_desc = database_features[database_name.split('/')[-1].split('.')[0]]
        orb = cv2.ORB_create(5000, 2.0)
#         keypoints_train, descriptors_train = orb.detectAndCompute(database, None)
#         keypoints_query, descriptors_query = orb.detectAndCompute(query, None)
        kp_logo, des_logo = orb.detectAndCompute(database, None)
        kp_img, des_img = orb.detectAndCompute(query, None)
        flann_index_lsh = 6
        index_params = dict(algorithm=flann_index_lsh,
                            table_number=12,
                            key_size=20,
                            multi_probe_level=2)
        search_params = dict(checks=100)  # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des_logo, des_img, k=2)
        print(len(matches))
        matches_mask = [[0, 0] for i in range(len(matches))]
        distances = list()
        counts = list()
        for i in range(len(matches)):            
            if len(matches[i]) == 2:
                m, n = matches[i]
#         for i, (m, n) in enumerate(matches):
                if m.distance < 0.7 * n.distance:
                    matches_mask[i] = [1, 0]
                    distances.append(m.distance)
        counts.append(len(distances))
        if len(distances) != 0:
            if np.max(counts) > max_count:
                max_count = np.max(counts)
                max_count_name = database_name.split('/')[-1].split('.')[0]
    print('[INFO] Max count: {} Database Image name: {} Query image name: {}'.format(max_count, max_count_name, query_name))
    if query_name.split('/')[-1].split('.')[0] == max_count_name:
        correct_count += 1
    else:
        print('[ERROR] database_image: {}, query_image: {}'.format(max_count_name, query_name.split('/')[-1].split('.')[0]))


2794
2794
2794
2794
[INFO] Max count: 2794 Database Image name: amstel_light Query image name: /home/garima/Desktop/clutter/Beer_images/query/amstel_light.jpg
756
756
756
756
[INFO] Max count: 756 Database Image name: amstel_light Query image name: /home/garima/Desktop/clutter/Beer_images/query/bira_blonde.jpg
[ERROR] database_image: amstel_light, query_image: bira_blonde
624
624
624
624
[INFO] Max count: 624 Database Image name: amstel_light Query image name: /home/garima/Desktop/clutter/Beer_images/query/becks_dark.jpg
[ERROR] database_image: amstel_light, query_image: becks_dark
2426
2426
2426
2426
[INFO] Max count: 2426 Database Image name: amstel_light Query image name: /home/garima/Desktop/clutter/Beer_images/query/becks.jpeg
[ERROR] database_image: amstel_light, query_image: becks


In [25]:
print('[INFO] Correct matches: {} total queries: {} Accuracy: {}'.format(correct_count, len(query_images), correct_count/len(query_images)))

[INFO] Correct matches: 1 total queries: 4 Accuracy: 0.25
